In [30]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import year,month,hour, when, col, date_format, to_timestamp, round, coalesce
from pyspark.sql import SparkSession
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

In [3]:
sc = pyspark.SparkSession('local', 'test')
#sc = pyspark.SparkContext(appName="Prediction model NO2")
sqlContext = SQLContext(sc)

C:\Users\baris\anaconda3\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
df = sqlContext.read.option("header",True).option("inferSchema",True) \
     .csv("pollution_us_2000_2016.csv")

In [5]:
df=df.withColumn("year",year("Date Local"))

In [6]:
df1=df.select("State","County","year","NO2 Mean","CO Mean",'O3 Mean','SO2 Mean')\
.groupBy("State","County","year").avg("NO2 Mean","CO Mean",'O3 Mean','SO2 Mean')

In [7]:
df1p=df1.toPandas()

In [8]:
df2p=pd.read_csv("death_data.csv")

In [9]:
us_state_abbrev={'AK':'Alaska','AL':'Alabama','AR':'Arkansas','AZ':'Arizona','CA': 'California','CO':'Colorado',\
                                 'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','IA':'Iowa',\
                                 'ID':'Idaho','IL': 'Illinois','IN':'Indiana','KS':'Kansas','KY':'Kentucky','LA':'Louisiana', \
                                'MA':'Massachusetts','MD':'Maryland','ME':'Maine','MI':'Michigan','MN':'Minnesota','MO':'Missouri', \
                                'NC':'North Carolina','ND':'North Dakota','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico', \
                                'NV':'Nevada','NY':'New York','OH':'Ohio','OK':'Oklahoma','OR':'Oregon','PA':'Pennsylvania' ,\
                                'RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota','TN':'Tennessee','TX':'Texas', \
                                'UT':'Utah','VA':'Virginia','WA':'Washington','WI':'Wisconsin','WY':'Wyoming'}

In [10]:
df2p['State']=df2p['State'].map(us_state_abbrev)

In [11]:
df_all=df1p.sort_values(by=['year'])

In [12]:
df3p=pd.merge(df_all,df2p,how="inner", left_on=['State','County','year'], right_on=['State','County_Name','Year'])

In [13]:
df3p=df3p.T.drop_duplicates().T

In [14]:
df4p=pd.read_csv("manufaturing_totals_2000_to_2016_restructure.csv")

In [15]:
df5p=df4p.dropna()

In [17]:
df5p['state']=df5p['state'].map(us_state_abbrev)

In [18]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [19]:
df5p = swap_columns(df5p, 'county', 'state')
df5p=df5p.rename(columns={"state": "State", "county": "County", "fat_amount": "num_factory"})

In [1]:
#df5p

In [21]:
df6p=pd.merge(df3p,df5p,how="inner", on=['State','County','year'])

In [2]:
#df6p

In [44]:
df7p=df6p[['avg(NO2 Mean)','avg(CO Mean)','avg(O3 Mean)','avg(SO2 Mean)', 'population_density','num_factory']]

In [101]:
#plt.hist(df6p['avg(NO2 Mean)'])
len(df6p[df6p['avg(NO2 Mean)']>9])/1094

0.696526508226691

In [40]:
cars=pd.read_csv("CA_county_cars.csv")

In [41]:
df7p_cars=pd.merge(df6p,cars,how="inner", on=['State','County','year'])

In [42]:
df8p=df7p_cars[['avg(NO2 Mean)','avg(CO Mean)','avg(O3 Mean)','avg(SO2 Mean)', 'population_density','num_factory','Autos','Trucks','Total Vehicles']]

In [45]:
df8p = df8p.astype(float)
df7p = df7p.astype(float)

In [46]:
#df7p.to_csv('df7.csv')
df7p.corr()

,avg(NO2 Mean),avg(CO Mean),avg(O3 Mean),avg(SO2 Mean),population_density,num_factory
avg(NO2 Mean),1.000000,0.587601,-0.525557,0.416436,0.437043,0.294672
avg(CO Mean),0.587601,1.000000,-0.392298,0.149995,0.186131,0.179794
avg(O3 Mean),-0.525557,-0.392298,1.000000,-0.170427,-0.336845,-0.160823
avg(SO2 Mean),0.416436,0.149995,-0.170427,1.000000,0.275931,-0.114577
population_density,0.437043,0.186131,-0.336845,0.275931,1.000000,0.047009
num_factory,0.294672,0.179794,-0.160823,-0.114577,0.047009,1.000000


In [47]:
df8p.corr()

,avg(NO2 Mean),avg(CO Mean),avg(O3 Mean),avg(SO2 Mean),population_density,num_factory,Autos,Trucks,Total Vehicles
avg(NO2 Mean),1.000000,0.511820,-0.105867,0.166962,0.257804,0.469287,0.594719,0.656627,0.613328
avg(CO Mean),0.511820,1.000000,-0.288117,0.128114,-0.044839,0.102574,0.145071,0.141606,0.143774
avg(O3 Mean),-0.105867,-0.288117,1.000000,-0.003403,-0.359067,-0.080305,-0.118654,-0.018385,-0.095352
avg(SO2 Mean),0.166962,0.128114,-0.003403,1.000000,0.019531,-0.139716,-0.004078,0.042580,0.006573
population_density,0.257804,-0.044839,-0.359067,0.019531,1.000000,0.502800,0.525918,0.437381,0.506228
num_factory,0.469287,0.102574,-0.080305,-0.139716,0.502800,1.000000,0.905554,0.848553,0.894844
Autos,0.594719,0.145071,-0.118654,-0.004078,0.525918,0.905554,1.000000,0.978361,0.998608
Trucks,0.656627,0.141606,-0.018385,0.042580,0.437381,0.848553,0.978361,1.000000,0.987782
Total Vehicles,0.613328,0.143774,-0.095352,0.006573,0.506228,0.894844,0.998608,0.987782,1.000000


In [48]:
train7,test7 = train_test_split(df7p, test_size=0.2, random_state=42)

In [49]:
X7_train = train7.drop(['avg(NO2 Mean)'], axis=1)
y7_train = train7['avg(NO2 Mean)']
X7_test = test7.drop(['avg(NO2 Mean)'], axis=1)
y7_test = test7['avg(NO2 Mean)']

In [50]:
p_model = LinearRegression()
p_model.fit(X7_train, y7_train)

predicted_no2 = p_model.predict(X7_test)

mean_squared_error(y7_test, predicted_no2)


11.79445004504477

In [51]:
mean_absolute_error(y7_test, predicted_no2)

2.639403969164844

In [52]:
mean_absolute_percentage_error(y7_test, predicted_no2)

0.35532939530478447

In [53]:
p_model.score(X7_test,y7_test)

0.6542914724927071

In [54]:
vif_data = pd.DataFrame() 
vif_data["feature"] = df7p.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(df7p.values, i) 
                          for i in range(len(df7p.columns))] 

vif_data

,feature,VIF
0,avg(NO2 Mean),10.669499
1,avg(CO Mean),6.246751
2,avg(O3 Mean),3.722601
3,avg(SO2 Mean),2.696611
4,population_density,1.615347
5,num_factory,1.558041


In [55]:
p_model1 = DecisionTreeRegressor(max_depth=5,random_state=7)
p_model1.fit(X7_train, y7_train)

predicted_no2 = p_model1.predict(X7_test)

mean_squared_error(y7_test, predicted_no2)

11.9876537155538

In [56]:
mean_absolute_error(y7_test, predicted_no2)

2.6962862943253434

In [57]:
mean_absolute_percentage_error(y7_test, predicted_no2)

0.3309806346074968

In [58]:
p_model11 = RandomForestRegressor(n_estimators=200,
                                  random_state=42)

p_model11.fit(X7_train, y7_train)

predicted_no22 = p_model11.predict(X7_test)

mean_squared_error(y7_test, predicted_no22)

6.166693468619689

In [59]:
mean_absolute_error(y7_test, predicted_no22)

1.8260032507756103

In [60]:
mean_absolute_percentage_error(y7_test, predicted_no22)

0.23482263904218015

In [61]:
param_grid = { 
    'n_estimators': [50,100,200],
    'random_state': [7,15,42],
    'oob_score':['True','False'],
    'criterion':['squared_error', 'absolute_error'] 
    
}


CV_rfc = GridSearchCV(estimator=p_model11, param_grid=param_grid, cv= 10)
CV_rfc.fit(X7_train, y7_train)

predicted_no33 = CV_rfc.predict(X7_test)

mean_squared_error(y7_test, predicted_no33)

6.244300211152868

In [63]:
mean_absolute_error(y7_test, predicted_no33)

1.8337115141124405

In [64]:
CV_rfc.best_params_

{'criterion': 'absolute_error',
 'n_estimators': 200,
 'oob_score': 'True',
 'random_state': 7}

In [65]:
train8,test8 = train_test_split(df8p, test_size=0.2, random_state=42)

In [66]:
X8_train = train8.drop(['avg(NO2 Mean)'], axis=1)
#X88_train = train8.drop(['avg(NO2 Mean)','avg(O3 Mean)','avg(SO2 Mean)'], axis=1)
y8_train = train8['avg(NO2 Mean)']
X8_test = test8.drop(['avg(NO2 Mean)'], axis=1)
#X88_test = test8.drop(['avg(NO2 Mean)','avg(O3 Mean)','avg(SO2 Mean)'], axis=1)
y8_test = test8['avg(NO2 Mean)']

In [67]:
X8_train

,avg(CO Mean),avg(O3 Mean),avg(SO2 Mean),population_density,num_factory,Autos,Trucks,Total Vehicles
67,0.252550,0.032934,0.772456,103.312778,8995.0,1096412.0,332313.0,1629237.0
12,0.810261,0.029360,0.555606,37.464079,116.0,88158.0,38116.0,147682.0
24,0.609601,0.027242,2.468413,676.371434,5846.0,1911703.0,483895.0,2653730.0
45,0.413190,0.022291,0.846364,459.266692,512.0,249543.0,68605.0,365427.0
107,0.266088,0.031561,0.131645,3330.034497,23310.0,2248999.0,400421.0,2831850.0
...,...,...,...,...,...,...,...,...
106,0.382242,0.025000,0.755633,1400.223142,2715.0,765068.0,144712.0,999733.0
14,0.685968,0.027398,2.504902,667.735179,6232.0,1899752.0,488794.0,2643283.0
92,0.406299,0.029338,0.243825,317.978562,7460.0,1279094.0,351311.0,1836206.0
51,0.413869,0.022778,0.534972,2080.768130,25548.0,5805760.0,1086927.0,7360573.0


In [68]:
y8_train

67     17.870176
12     14.148143
24     17.324475
45     10.211051
107    11.617735
         ...    
106     7.198159
14     19.312337
92     15.181102
51     19.956466
102    13.987552
Name: avg(NO2 Mean), Length: 99, dtype: float64

In [69]:
p_model2 = LinearRegression()
p_model2.fit(X8_train, y8_train)

predicted_no22 = p_model2.predict(X8_test)

mean_squared_error(y8_test, predicted_no22)

8.284689383518991

In [70]:
mean_absolute_error(y8_test, predicted_no22)

2.347262469146624

In [71]:
mean_absolute_percentage_error(y8_test, predicted_no22)

0.4107180707277525

In [72]:
p_model2.score(X8_test,y8_test)

0.726482192118924

In [73]:
p_model1 = DecisionTreeRegressor(max_depth=None,random_state=7)
p_model1.fit(X8_train, y8_train)

predicted_no2 = p_model1.predict(X8_test)

mean_squared_error(y8_test, predicted_no2)

2.991498831787429

In [74]:
mean_absolute_error(y8_test, predicted_no2)

1.2956858773328905

In [75]:
mean_absolute_percentage_error(y8_test, predicted_no2)

0.16365646573027717

In [76]:
p_model11 = RandomForestRegressor(n_estimators=100,
                                  random_state=75)

p_model11.fit(X8_train, y8_train)

predicted_no22 = p_model11.predict(X8_test)

mean_squared_error(y8_test, predicted_no22)

3.0284058867901598

In [77]:
mean_absolute_error(y8_test, predicted_no22)

1.243704678155003

In [78]:
mean_absolute_percentage_error(y8_test, predicted_no22)

0.2673998733489899

In [89]:
len(y8_test[y8_test>8.5])/len(y8_test)

0.8

In [100]:
len(df6p[(df6p['State']=='California')&(df6p['avg(NO2 Mean)']>6)])/len(df6p[df6p['State']=='California'])

0.8382978723404255

In [151]:
param_grid = { 
    'n_estimators': [50,100,200],
    'random_state': [7,15,42],
    'oob_score':['True','False'],
    'criterion':['absolute_error'] 
    
}


CV_rfc = GridSearchCV(estimator=p_model11, param_grid=param_grid, cv= 10)
CV_rfc.fit(X8_train, y8_train)

predicted_no33 = CV_rfc.predict(X8_test)

mean_squared_error(y8_test, predicted_no33)

3.2141117619159165

In [152]:
mean_absolute_error(y8_test, predicted_no33)

1.3296281860822128

In [153]:
mean_absolute_percentage_error(y8_test, predicted_no22)

0.27697106835340496